In [ ]:
# [Nur Colab] Diese Zellen müssen nur auf *Google Colab* ausgeführt werden und installieren Packete und Daten
!wget -q https://raw.githubusercontent.com/KI-Campus/AMALEA/master/requirements.txt && pip install --quiet -r requirements.txt
!wget --quiet "https://github.com/KI-Campus/AMALEA/releases/download/data/data.zip" && unzip -q data.zip
!wget --quiet "https://raw.githubusercontent.com/KI-Campus/AMALEA/master/Woche%205/utils.py"

# Datenmangel? Copy & augmentated Paste

## CNN Übung
In diesem Notebook werden Sie mehrschichtige Perzeptronen (MLP, engl. Multilayer Perceptrons) mit Convolutional Neural Networks (CNN) vergleichen, um eine einfache Aufgabe wie die Erkennung von 10 verschiedenen Objekten aus Bildern zu lösen. Sie werden alles mit Keras programmieren und die Ergebnisse mit den bereits gelernten Metriken für die Klassifizierung vergleichen.

### Importe

In [1]:
from ipywidgets import widgets
from IPython.display import display

import matplotlib.pyplot as plt
import numpy as np

# For some convolving operations
from scipy import signal
from scipy import misc

# DeepLearning Library Keras
# Documentation https://keras.io/
import tensorflow.keras as keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, BatchNormalization, Dropout, Reshape
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.datasets import cifar10

import utils

#define dataroot
root = 'data/dataset/'


### Definitionen

In [2]:
#### DO NOT EDIT!
# For specifiying training with autoencoder structure
ae_specification = widgets.Text()
old_spec = 'None'


# Two Loggers, depending if loss or accuracy should be visualized
Loss_Logger = utils.LossGraph('loss')
Acc_Logger = utils.LossGraph('acc')

# Size for some plots with matplotlib
figure_inches = 3

### Cifar-10 Klassifikationsaufgabe
Neben dem MNIST-Datensatz ist auch Cifar10 ein kleiner Datensatz, der in den Anfängen der CNNs verwendet wurde. Es gibt 10 verschiedene Klassen von einfachen Objekten oder Tieren. Die Bilder haben eine Größe von 32x32x3. In diesem Abschnitt sollten Sie ein gegebenes CNN tunen, um Bilder mit hoher Genauigkeit zu klassifizieren. 

Siehe auch: [Cifar-10](https://www.cs.toronto.edu/~kriz/cifar.html)


<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.1:</b> Laden Sie den Datensatz, legen Sie die Anzahl der Klassen fest, transformieren Sie die Beschriftungen und definieren Sie alle zugehörigen Klassen (wie z.B. Flugzeug,...) gemäß den Kommentaren in den Codezellen.

</div>

In [3]:
# Load the dataset from Keras, Tip: cifar10 is already imported, train + test set required
# HINT: Load the dataset into this structure: "(x_train, y_train), (x_test, y_test) = ..."

# STUDENT CODE HERE

# STUDENT CODE until HERE

In [4]:
# How many classes are in Cifar-10? 
# Hint: Name the variable "num_classes = ..."

# STUDENT CODE HERE

# STUDENT CODE until HERE

In [5]:
# Transform the labels into categorical vectors
# Use the keras.utils.to_categorical function

# Hint: "y_train_categorical = ..."
# Hint2: "y_test_categorical = ..."

# STUDENT CODE HERE

# STUDENT CODE until HERE

In [6]:
# What classes are there? Define them in a list of strings named classes.
# Hint: Call the list of strings "classes = ..."

# STUDENT CODE HERE

# STUDENT CODE until HERE

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.2:</b> Prüfen Sie, ob Sie alles wie gewünscht definiert haben.

</div>

In [7]:
# Image in the training set
number_sample = 6  # test multiple ones

fig, ax = plt.subplots(figsize=(figure_inches, figure_inches))
ax.set_title(classes[y_test[number_sample].item()]+' ?', fontsize = 15)
ax.imshow(x_test[number_sample,:,:,:], interpolation='nearest')
plt.tight_layout()

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.3:</b> Suchen Sie ein Bild eines Pferdes und plotten Sie es mit dem Code oben. Sie können die gleiche Code-Zelle verwenden.

</div>

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.4:</b> Preprocessen Sie die Daten, um Werte zwischen 0 und 1 zu gewährleisten. Dividieren Sie dazu die rgb-Werte durch ihren Maximalwert.

</div>

In [8]:
# Data Preprocessing
# Divide RGB values of train AND test set 
# by their maximum value to ensure values between [0,1]

# STUDENT CODE HERE

# STUDENT CODE until HERE

<div class="alert alert-block alert-success">
<b>Frage 5.3.5:</b> Wie viele Trainings- und Testdaten gibt es?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.3.6:</b> Warum Daten normalisieren?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.3.7:</b> Warum wird ein kategorischer Vektor an Stelle eines einzelnen Outputs verwendet?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


## Klassifikationsmodelle für Cifar-10

### Neuronales Netzwerk Klassifikator:

In [9]:
def model_nn()->Model:
    
    input_layer = Input(shape = x_train.shape[1:], name='Input_MLP')
    
    flatten = Flatten(name='Flattening_MLP')(input_layer)
    
    hidden1 = Dense(256, activation = 'relu', name='Hidden1_NN')(flatten)
    hidden2 = Dense(256, activation = 'relu', name='Hidden2_NN')(hidden1)
    
    output = Dense(num_classes, activation = 'softmax', name='Output_NN')(hidden2)
    
    model = Model(inputs= input_layer, outputs = output)
    
    
    return model

### Convolutional Neural Network Klassifikator:

In [10]:
def model_cnn()->Model:
    
    input_layer = Input(shape = x_train.shape[1:], name='Input_CNN') # channels last
    
    conv1 = Conv2D(filters= 16, kernel_size = (3,3), padding='same', activation = 'relu', name='Conv1')(input_layer)
    max_pool1 = MaxPooling2D(pool_size = (2,2), strides = (2,2), padding='valid', name='Pool1')(conv1)

    conv2 = Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu', name='Conv2')(max_pool1)
    max_pool2 = MaxPooling2D(pool_size = (2,2), strides = (2,2), padding='valid', name='Pool2')(conv2)

    flattened = Flatten(name='Flatt_CNN')(max_pool2)
    
    fc1 = Dense(256, activation = 'relu', name='FC-1')(flattened)
    
    output = Dense(num_classes, activation = 'softmax', name='Output_CNN')(fc1)
    
    model = Model(inputs= input_layer, outputs = output)
        
    return model

### Vergleich von MLP und CNN Klassifikatoren:

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.8:</b> Um die Bilder in Cifar-10 zu klassifizieren, verwenden Sie die gegebenen MLP- und CNN-Modelle, um zu untersuchen, welches besser abschneidet.
Trainieren Sie beide Netzwerke für 10 Epochen und schauen Sie sich die Ergebnisse an.
Fühlen Sie sich frei, den Code in den beiden Code-Zellen unten zu verwenden und zu ändern. Wenn Ihr Netzwerk nicht trainiert, haben Sie möglicherweise die rgb-Werte nicht richtig aufbereitet (z. B. haben Sie nicht normalisiert oder zu oft).
</div>

<div class="alert alert-block alert-info">
<b>Hinweis:</b> Aufbau der folgenden Codezellen
<ul>
<li> Benutzen Sie die vordefinierten Funktionen, um Ihr Modell zu erstellen
<li> Definieren Sie den gemeinsamen TensorBoard-Logger mit der Konfiguration, um die Trainingsergebnisse später zu betrachten
<li> Kompillieren und trainieren Sie das Modell
<li> Tipp: Wenn Ihre Modelle nichts Lernen, überprüfen Sie Ihre Datennormalisierung. Vielleicht haben Sie Ihre Daten nicht oder zu oft normalisiert.
</li>


</ul>


</div>

In [11]:
 # Train the Neural Network (MLP)
nn_model = model_nn()
config_nn = 'UNRECOGNIZEABLE_NAME_EDIT_ME_PLEASE' # Give a recognizable name

# The TensorBoard is a feature of tensorflow for the visualization of the training process 
nn_logger = TensorBoard(log_dir='logs/nn_logs/'+config_nn+'/')

nn_model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='Adam')
nn_model.fit(x_train, y_train_categorical, batch_size = 32, epochs = 1, 
            validation_data = (x_test, y_test_categorical), callbacks = [nn_logger, Acc_Logger], verbose = 1) #TODO

In [12]:
# Train the Neural Network (MLP)
nn_model = model_nn()
config_cnn = 'UNRECOGNIZEABLE_NAME_EDIT_ME_PLEASE' # Give a recognizable name

# The TensorBoard is a feature of tensorflow for the visualization of the training process 
cnn_logger = TensorBoard(log_dir='logs/cnn_logs/'+config_cnn+'/') 

nn_model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='Adam')
nn_model.fit(x_train, y_train_categorical, batch_size = 64, epochs = 1, 
            validation_data = (x_test, y_test_categorical), callbacks = [cnn_logger, Acc_Logger]) #TODO

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.9:</b> Verwenden Sie TensorBoard, um Ihren Trainingsfortschritt zu kontrollieren. Eine Erklärung, wie Sie Ihr TensorBoard öffnen, finden Sie hier:
    <a href="https://github.com/tensorflow/tensorboard/blob/master/docs/r1/summaries.md">TensorBoard</a>  (unten auf der Webseite)
</div>

In [13]:
# Train the CNN
%load_ext tensorboard
cnn_model = model_cnn()
config_cnn = 'UNRECOGNIZEABLE_NAME_EDIT_ME_PLEASE' # give a recognizable name

cnn_model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='Adam')
cnn_model.fit(x_train, y_train_categorical, batch_size = 32, epochs = 1, 
          validation_data = (x_test, y_test_categorical), callbacks = [Acc_Logger], verbose = 1)
%tensorboard --logdir logs

<div class="alert alert-block alert-success">
<b>Frage 5.3.10:</b> Welches Netzwerk performt besser?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.3.11:</b> Wie viele Parameter haben die Netze? Verwenden Sie dazu die summary Methode (siehe Keras-Docs)...
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.3.12:</b> Wo sind die meisten Parameter in diesem CNN gespeichert?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


#### Challenge: Optimieren Sie Ihr Network! 

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.13:</b> Versuchen Sie, eines der Modelle so zu verbessern, dass Ihre Validierungsgenauigkeit einmal höher als 0,75 Prozent ist!

<ul>
<li>Hinweis: Versuchen Sie, zuerst zu overfitten und dann zu regulieren. 
<li>Hinweis 2: Verwenden Sie daher L1/L2 - Regularisierung und/oder Dropout. Auch BatchNormalization könnte die Sache verbessern. Schauen Sie deshalb auf der Keras-Website nach Beispielen oder fragen Sie Tutoren.
<li>Hinweis 3: Verwenden Sie eine der Funktionen <code>def model_nn()</code> oder <code>def model_cnn()</code> von oben. Viel Spaß und gutes Gelingen!

</li>
</ul>
</div>

### Daten Augmentation

Eine weitere Möglichkeit, Ihr Netzwerk zu regularisieren, ist das Vergrößern der Trainingsdaten. Verwenden Sie dazu den ImageDataGenerator von Keras. Wir werden später selbst Bilder verschieben und drehen, nachdem wir auf Cifar-10 optimiert haben.

In [14]:
# Keras ImageDataGenerator
datagen = ImageDataGenerator(
    featurewise_center=False,
    featurewise_std_normalization=False,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

#### Eine etwas noch herausfordernde Challenge ((COOKIE AUFGABE! :)))

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.14:</b> Verbessern Sie Ihr Modell und passen Sie es an, wie genau kann es jetzt werden?
Die Lösung ist in der Lage, eine Genauigkeit von 0,8894 bei der Validierung zu erreichen.
</div>

In [15]:
def model_cnn_aug()->Model:

    input_layer = Input(shape = x_train.shape[1:], name='Input_CNN') # channels last
    
    conv1 = Conv2D(filters= 16, kernel_size = (3,3), padding='same', activation = 'relu', name='Conv1')(input_layer)
    max_pool1 = MaxPooling2D(pool_size = (2,2), strides = (2,2), padding='valid', name='Pool1')(conv1)

    conv2 = Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu', name='Conv2')(max_pool1)
    max_pool2 = MaxPooling2D(pool_size = (2,2), strides = (2,2), padding='valid', name='Pool2')(conv2)

    flattened = Flatten(name='Flatt_CNN')(max_pool2)
    
    fc1 = Dense(256, activation = 'relu', name='FC-1')(flattened)
    
    output = Dense(num_classes, activation = 'softmax', name='Output_CNN')(fc1)
    
    model = Model(inputs= input_layer, outputs = output)
    
    
    return model

In [16]:
# Train your model that makes use of data augmentation
# Fit the training data to the data-generator
datagen.fit(x_train)

# Train your CNN_augmentation model:
cnn_aug_model = model_cnn_aug()
cnn_aug_model.compile(loss='categorical_crossentropy', metrics = ['accuracy'], optimizer='Adam')
config_cnn_aug = 'None' # give a recognizable name
cnn_logger = TensorBoard(log_dir='logs/cnn_aug_logs/'+config_cnn_aug+'/') 

cnn_aug_model.fit(datagen.flow(x_train, y_train_categorical, batch_size = 5),
                            epochs=1, validation_data=(x_test, y_test_categorical),
                            callbacks=[Acc_Logger, cnn_logger], verbose=1, steps_per_epoch=10000)


<div class="alert alert-block alert-success">
<b>Frage 5.3.15:</b> Können Sie sich vorstellen, warum die Labels in der Codezelle oben nicht erweitert wurden und ob das notwendig sein könnte? Wenn Ihnen die Intuition fehlt, können Sie auf diese Frage zurückkommen, nachdem Sie das Notebook oder die Implementierung der Datenerweiterung unten abgeschlossen haben. 
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


<div class="alert alert-block alert-success">
<b>Frage 5.3.16:</b> Was denken Sie, was mit den vom DataGenerator angepassten Bildern passiert?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


### Sagen Sie mit Ihrem Model vorher

In [17]:
# Get the probabilities of one image prediction
x_tester = x_test[1,:,:,:]  

# Use numpy expand_dims before predicting with your model
# Print your predicted classes for the first test image (x_test[0,:,:,:])

# STUDENT CODE HERE

# STUDENT CODE until HERE


<div class="alert alert-block alert-success">
<b>Frage 5.3.17:</b> Mit wie viel Konfidenz wurde Bild <b>18</b> in der Testmenge von Ihrem Modell als Vogel vorhergesagt?
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>


### Schauen wir genauer hin / Ermitteln Sie die Gewichte in einer Faltungsschicht (engl. convolutional layer)

In [18]:
# Get the weights of a layer of one of your models, you specified by name
layer_visual = cnn_model.get_layer('Conv1') 
weights = layer_visual.get_weights()[0]

# Take some of them, last dimension are the channels
weights_2d = weights[:,:,0,0] # filters are [:,:, dimension of spatial input (e.g.: rgb=3), nb_filters] in a layer
weights_2d

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.18:</b> Schauen Sie sich die Schicht Conv2 (oder eine andere Schicht als Conv1) an und zeichnen Sie eine Filter-Kernel-Scheibe ihres dritten Filters. Tipp: Verwenden Sie weights.shape, um die Dimensionen des Kernels zu verstehen.
</div>

<div class="alert alert-block alert-success">
<b>Frage 5.3.19:</b> Erklären Sie, was die Dimensionen a,b,c und d in 'weights[a,b,c,d]' sind, wie es im obigen Codeblock verwendet wird.
</div>

<div class="alert alert-block alert-success">
<b>Ihre Antwort:</b></div>










In [19]:
#Load Ascent image from scipy and convolve it with the previous loaded filter
ascent = misc.ascent()
ascent = signal.convolve2d(ascent, weights_2d, boundary='symm', mode='same')
ascent = np.maximum(ascent, 0)
fig, ax = plt.subplots(figsize=(figure_inches, figure_inches))
ax.imshow(ascent, interpolation='nearest', cmap='gray')
plt.tight_layout()

<div class="alert alert-block alert-success">
<b>Aufgabe 5.3.20:</b> Verwenden Sie verschiedene Filter auf das Inputbild. Können Sie irgendwelche Unterschiede feststellen? (Ein paar Worte)
</div>

### Visualisierung von Aktivierung in einem Feedforward-Durchlauf 

Im folgenden Code werden wir direkt die Ausgabe der Faltungsschicht im CNN verwenden und visualisieren. Dies entspricht in etwa dem, was wir oben gemacht haben.  

In [20]:
# Get the output in a feedforward process from a model with get_output function
number_sample = 9

# Model and layer where the feature maps come from
feature_map = utils.get_output(cnn_model, 'Conv1', np.expand_dims(x_test[number_sample,:,:,:],axis=0))

# Take only 32 filters of the layer if there are so many
feature_map = feature_map[:,:,:,:32]

plt.imshow(x_test[number_sample,:,:,:])
plt.title(classes[y_test[number_sample].item()])
plt.subplots(figsize=(15, 15))

num_columns = 4
num_rows = 8
for i in range(0,feature_map.shape[-1]):
    
    plt.subplot(num_rows, num_columns, i+1)
    plt.imshow(feature_map[0,:,:,i], cmap ='gray')
    plt.axis('off')